In [1]:
import requests
import json
from pprint import pprint
from datetime import datetime

In [2]:
def get_token():
    oauth = "y0_AgAAAAAdy8jaAATuwQAAA8SG3Xh2A"
    url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
    headers = {
        "Content-Type": "application/json"
    }
    data=json.dumps({"yandexPassportOauthToken": oauth})
    res = requests.post(url, data=data, headers=headers)
    return res.json()['iamToken']

In [3]:
def get_answer(system, user, temp):
    folder = 'b1gqlhmqdst9ejbptsnp'
    temperature = temp
    maxTokens = "1000"
    system_message = system
    user_message = user
    
    body = {
      "modelUri": f"gpt://{folder}/yandexgpt/latest",
      "completionOptions": {
        "stream": False,
        "temperature": temperature,
        "maxTokens": maxTokens
      },
      "messages": [
        {
          "role": "system",
          "text": system_message
        },
        {
          "role": "user",
          "text": user_message
        }
      ]
    }
    
    url = 'https://llm.api.cloud.yandex.net/foundationModels/v1/completion'
    
    headers = {
        "Authorization": f"Bearer {get_token()}",
        "Content-Type": "application/json"
    }
    data = json.dumps(body)
    
    res = requests.post(url, data=data, headers=headers)

    return res.json()['result']['alternatives'][0]['message']['text']

In [5]:
imp = [-0.968, -1.302, 2.335, 1.673, -0.499, 0.88, -0.823, 1.611]
imp = [round(x, 1) for x in imp]
high_left = min(imp)
high_right = max(imp)

low_left = max([x for x in imp if x < 0])
low_right = min([x for x in imp if x > 0])

mid_left = round((high_left + low_left) / 2, 2)
mid_right = round((high_right + low_right) / 2, 2)

# 

# Ответь в формате json слещующего образца {{"HIGH": <Список правил>, "MID": <Список правил>, "LOW": <Список правил>}}

system = f"""
Ты опытный аналитик данных, который занимается поиском зависимости значений Переменных (Sex, Pclass, Age) от переменной Importance.
Дерево решений, которое ты обучил, выдало тебе набор правил. 
Категоризируй правила по следующим категориям:
HIGH - Значения Importance принадлежит отрезку [{high_left}, {mid_left}) или отрезку ({mid_right}, {high_right}];
MID - Значения Importance принадлежит отрезку [{mid_left}, {low_left}) или отрезку ({low_right}, {mid_right}];
LOW - Значения Importance принадлежит отрезку [{low_left}, {low_right}];
В ответ выведи список категорий со списком правил, которые к ним относятся.
Не используй таблицы.
"""

print("Promt", system)
print('\n\n')

user = """
1. Если (Sex <= 0.5) И (Pclass <= 2.5) И (Age > 10.5) то Importance равна 2.33;
2. Если (Sex <= 0.5) И (Pclass > 2.5) И (Age > 12.0) то Importance равна 1.67;
3. Если (Sex <= 0.5) И (Pclass <= 2.5) И (Age <= 10.5) то Importance равна 1.61;
4. Если (Sex > 0.5) И (Pclass <= 2.5) И (Age > 13.5) то Importance равна -1.3;
5. Если (Sex > 0.5) И (Pclass > 2.5) И (Age > 13.0) то Importance равна -0.97;
6. Если (Sex <= 0.5) И (Pclass > 2.5) И (Age <= 12.0) то Importance равна 0.88;
7. Если (Sex > 0.5) И (Pclass <= 2.5) И (Age <= 13.5) то Importance равна -0.82;
8. Если (Sex > 0.5) И (Pclass > 2.5) И (Age <= 13.0) то Importance равна -0.5;
"""
temp = 0.4

print(get_answer(system, user, temp))

# for _ in range(5):
#     print(_)
#     print(get_answer(system, user, temp))
#     print('\n\n\n\n\n')


Promt 
Ты опытный аналитик данных, который занимается поиском зависимости значений Переменных (Sex, Pclass, Age) от переменной Importance.
Дерево решений, которое ты обучил, выдало тебе набор правил. 
Категоризируй правила по следующим категориям:
HIGH - Значения Importance принадлежит отрезку [-1.3, -0.9) или отрезку (1.6, 2.3];
MID - Значения Importance принадлежит отрезку [-0.9, -0.5) или отрезку (0.9, 1.6];
LOW - Значения Importance принадлежит отрезку [-0.5, 0.9];
В ответ выведи список категорий со списком правил, которые к ним относятся.
Не используй таблицы.




**HIGH**:
* Правило 1: Если (Sex <= 0.5) И (Pclass <= 2.5) И (Age > 10.5), то Importance равна 2.33;
* Правило 2: Если (Sex <= 0.5) И (Pclass > 2.5) И (Age > 12.0), то Importance равна 1.67.

**MID**:
* Правило 3: Если (Sex <= 0.5) И (Pclass <= 2.5) И (Age <= 10.5), то Importance равна 1.61;
* Правило 6: Если (Sex <= 0.5) И (Pclass > 2.5) И (Age <= 12.0), то Importance равна 0.88.

**LOW**:
* Правило 4: Если (Sex > 0.5) 

In [6]:
system = f"""
Ты опытный аналитик данных, который занимается поиском зависимости значений Переменных (Sex, Pclass, Age) от переменной Importance.
Дерево решений, которое ты обучил, выдало тебе набор правил. 
Категоризируй правила по значению переменной Importance, должны быть три категории - высокая, средняя и низкая важность
В ответ выведи список категорий со списком правил, которые к ним относятся.
Не используй таблицы.
"""

print("Promt", system)
print('\n\n')

user = """
1. Если (Sex <= 0.5) И (Pclass <= 2.5) И (Age > 10.5) то Importance равна 2.33;
2. Если (Sex <= 0.5) И (Pclass > 2.5) И (Age > 12.0) то Importance равна 1.67;
3. Если (Sex <= 0.5) И (Pclass <= 2.5) И (Age <= 10.5) то Importance равна 1.61;
4. Если (Sex > 0.5) И (Pclass <= 2.5) И (Age > 13.5) то Importance равна -1.3;
5. Если (Sex > 0.5) И (Pclass > 2.5) И (Age > 13.0) то Importance равна -0.97;
6. Если (Sex <= 0.5) И (Pclass > 2.5) И (Age <= 12.0) то Importance равна 0.88;
7. Если (Sex > 0.5) И (Pclass <= 2.5) И (Age <= 13.5) то Importance равна -0.82;
8. Если (Sex > 0.5) И (Pclass > 2.5) И (Age <= 13.0) то Importance равна -0.5;
"""
temp = 0.4

print(get_answer(system, user, temp))

for _ in [0, 0.2, 0.4, 0.6]:
    print(_)
    print(get_answer(system, user, temp))
    print('\n\n\n\n\n')


Promt 
Ты опытный аналитик данных, который занимается поиском зависимости значений Переменных (Sex, Pclass, Age) от переменной Importance.
Дерево решений, которое ты обучил, выдало тебе набор правил. 
Категоризируй правила по значению переменной Importance, должны быть три категории - высокая, средняя и низкая важность
В ответ выведи список категорий со списком правил, которые к ним относятся.
Не используй таблицы.




**Высокая важность**:
* Правило 1: если (Sex <= 0.5) И (Pclass <= 2.5) И (Age > 10.5), то Importance равна 2.33.

**Средняя важность**:
* Правило 6: если (Sex <= 0.5) И (Pclass > 2.5) И (Age <= 12.0), то Importance равна 0.88.

**Низкая важность**:
* Правило 3: если (Sex <= 0.5) И (Pclass <= 2.5) И (Age <= 10.5), то Importance равна 1.61.
* Правило 6 можно также отнести к этой категории.

Остальные правила относятся к категориям высокой или средней важности.

*Примечание: в зависимости от того, как именно вы определяете категории важности, результат может отличаться.*
0


In [ ]:
print(user)


In [7]:
system = f"""
Ты опытный аналитик данных, который занимается поиском зависимости значений переменных.
Дерево решений, которое ты обучил, выдало тебе набор правил. 
Категоризируй правила по значению переменной Importance, должны быть три категории - высокая, средняя и низкая абсолютная важность.
Способ категоризации - проверка абсолютного значения переменной I.
В ответ выведи список категорий со списком правил, которые к ним относятся.
Не используй таблицы.
"""

print("Promt", system)
print('\n\n')

user = """
1. Если (Sex <= 0.5) И (Pclass <= 2.5) И (Age > 10.5) то Importance равна 2.33;
2. Если (Sex <= 0.5) И (Pclass > 2.5) И (Age > 12.0) то Importance равна 1.67;
3. Если (Sex <= 0.5) И (Pclass <= 2.5) И (Age <= 10.5) то Importance равна 1.61;
4. Если (Sex > 0.5) И (Pclass <= 2.5) И (Age > 13.5) то Importance равна -1.3;
5. Если (Sex > 0.5) И (Pclass > 2.5) И (Age > 13.0) то Importance равна -0.97;
6. Если (Sex <= 0.5) И (Pclass > 2.5) И (Age <= 12.0) то Importance равна 0.88;
7. Если (Sex > 0.5) И (Pclass <= 2.5) И (Age <= 13.5) то Importance равна -0.82;
8. Если (Sex > 0.5) И (Pclass > 2.5) И (Age <= 13.0) то Importance равна -0.5;
""".replace("Importance", "I").replace("Sex", "A").replace("Pclass", "B").replace("Cabin", "C").replace("Cabin", "D").replace("Age", "E").replace("Fare", "F")

temp = 0.3

print(get_answer(system, user, temp))

# for _ in [0.3, 0.6, 0.8]:
#     print(_)
#     print(get_answer(system, user, _))
#     print('\n\n\n\n\n')


Promt 
Ты опытный аналитик данных, который занимается поиском зависимости значений переменных.
Дерево решений, которое ты обучил, выдало тебе набор правил. 
Категоризируй правила по значению переменной Importance, должны быть три категории - высокая, средняя и низкая абсолютная важность.
Способ категоризации - проверка абсолютного значения переменной I.
В ответ выведи список категорий со списком правил, которые к ним относятся.
Не используй таблицы.




**Высокая абсолютная важность**:
* Правило 4: если (A > 0.5) И (B <= 2.5) И (E > 13.5), то I равна -1.3.
* Правило 5: если (A > 0.5) И (B > 2.5) И (E > 13.0), то I равна -0.97.

**Средняя абсолютная важность**:
* Правило 2: если (A <= 0.5) И (B > 2.5) И (E > 12.0), то I равна 1.67.
* Правило 6: если (A <= 0.5) И (B > 2.5) И (E <= 12.0), то I равна 0.88.
* Правило 8: если (A > 0.5) И (B > 2.5) И (E <= 13.0), то I равна -0.5.

**Низкая абсолютная важность**:
* Правило 1: если (A <= 0.5) И (B <= 2.5) И (E > 10.5), то I равна 2.33.
* Правил

In [ ]:
проанализируй правила из полученного набора и определи, какие из них наиболее важны для понимания влияния переменных датасета на изучаемую переменную в контексте модели классификации выживаемости оборудования
Правила:
1. Sex <= 0.5 & Pclass <= 2.5 -> Importance=2.294
2. Sex <= 0.5 & Pclass > 2.5 -> Importance=1.495
3. Sex > 0.5 & Pclass <= 2.5 -> Importance=-1.273
4. Sex > 0.5 & Pclass > 2.5 -> Importance=-0.921

In [ ]:
Опиши параметры оборудования,с повышенным шансом на выживание